In [1]:
from scipy import stats
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from dask_ml.model_selection import train_test_split
import joblib
import dask.array as da
import dask.dataframe as dd
from dask_ml.preprocessing import DummyEncoder
from dask_ml.linear_model import LinearRegression

from dask.distributed import Client, progress

client = Client(n_workers=4, threads_per_worker=2, memory_limit='2GB')
client

/opt/anaconda3/lib/python3.7/site-packages/dask/array/random.py:27: FutureWarning: dask.array.random.doc_wraps is deprecated and will be removed in a future version
  FutureWarning,


Client Scheduler: tcp://127.0.0.1:54397 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 8.00 GB


In [2]:
cars_df = dd.read_csv('/Users/steveangeli/Downloads/vehicles.csv')

In [3]:
cars_df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 25 entries, id to long
dtypes: object(18), float64(5), int64(2)

In [4]:
cars_df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,drive,size,type,paint_color,image_url,description,county,state,lat,long
0,7034441763,https://saltlakecity.craigslist.org/cto/d/salt...,salt lake city,https://saltlakecity.craigslist.org,17899,2012.0,volkswagen,golf r,excellent,4 cylinders,...,4wd,compact,hatchback,black,https://images.craigslist.org/00G0G_fTLDWM5Xyv...,PRICE REDUCED! -Garage kept -Low Miles (63K)...,NaN,ut,40.7372,-111.858
1,7034440610,https://saltlakecity.craigslist.org/ctd/d/sand...,salt lake city,https://saltlakecity.craigslist.org,0,2016.0,ford,f-150,excellent,NaN,...,4wd,NaN,NaN,NaN,https://images.craigslist.org/00v0v_7Cu0buIofU...,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
2,7034440588,https://saltlakecity.craigslist.org/ctd/d/sand...,salt lake city,https://saltlakecity.craigslist.org,46463,2015.0,gmc,sierra 1500,excellent,NaN,...,4wd,NaN,NaN,white,https://images.craigslist.org/01515_lPvJ9bfbdY...,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
3,7034440546,https://saltlakecity.craigslist.org/ctd/d/sand...,salt lake city,https://saltlakecity.craigslist.org,0,2016.0,ford,f-150,excellent,NaN,...,4wd,NaN,NaN,NaN,https://images.craigslist.org/00T0T_6Rjfp3NS4O...,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
4,7034406932,https://saltlakecity.craigslist.org/ctd/d/evan...,salt lake city,https://saltlakecity.craigslist.org,49999,2018.0,ford,f-450,NaN,NaN,...,4wd,NaN,pickup,white,https://images.craigslist.org/00W0W_8yIUwRBXXd...,2018 Ford F-350 F350 F 350 SD Lariat Crew Cab ...,NaN,ut,40.3744,-104.694


In [5]:
print(cars_df['manufacturer'].value_counts().compute())
print(cars_df['condition'].value_counts().compute())
print(cars_df['cylinders'].value_counts().compute())
print(cars_df['fuel'].value_counts().compute())
print(cars_df['title_status'].value_counts().compute())
print(cars_df['transmission'].value_counts().compute())
print(cars_df['drive'].value_counts().compute())
print(cars_df['size'].value_counts().compute())
print(cars_df['type'].value_counts().compute())
print(cars_df['paint_color'].value_counts().compute())

ford               92601
chevrolet          73661
toyota             38159
nissan             26435
ram                26177
honda              25585
jeep               23951
gmc                23126
dodge              19344
bmw                13468
hyundai            12182
mercedes-benz      11569
subaru             10259
volkswagen         10214
chrysler            8866
kia                 8821
cadillac            7304
buick               6357
lexus               6292
mazda               5851
audi                5586
acura               4169
infiniti            3933
pontiac             3493
lincoln             3393
volvo               3088
mitsubishi          2848
mini                2173
rover               1840
saturn              1790
mercury             1760
jaguar              1029
fiat                 746
harley-davidson      215
tesla                207
alfa-romeo           109
ferrari               82
datsun                68
land rover            25
aston-martin          22


In [6]:
# Drop some columns that I don't believe wil have much impact on the sale price or that have too many values.
cars_df = cars_df.drop(columns=['url', 'manufacturer', 'model', 'region_url', 'vin', 'image_url', 'description', 'county', 'lat', 'long', 'id', 'region', 'state', 'model', 'paint_color', 'type', 'cylinders'])



In [7]:
len(cars_df)

509577

In [8]:
cars_df.head()

,price,year,condition,fuel,odometer,title_status,transmission,drive,size
0,17899,2012.0,excellent,gas,63500.0,clean,manual,4wd,compact
1,0,2016.0,excellent,gas,10.0,clean,automatic,4wd,NaN
2,46463,2015.0,excellent,gas,7554.0,clean,automatic,4wd,NaN
3,0,2016.0,excellent,gas,10.0,clean,automatic,4wd,NaN
4,49999,2018.0,NaN,diesel,70150.0,clean,automatic,4wd,NaN


In [9]:
# Drop any null rows because I still should have plenty of data to work with. 
cars_df = cars_df.dropna()
cars_df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 9 entries, price to size
dtypes: object(6), float64(2), int64(1)

In [10]:
len(cars_df)

121500

In [11]:
print(cars_df['condition'].value_counts().compute())
print(cars_df['fuel'].value_counts().compute())
print(cars_df['title_status'].value_counts().compute())
print(cars_df['transmission'].value_counts().compute())
print(cars_df['drive'].value_counts().compute())
print(cars_df['size'].value_counts().compute())

excellent    62184
good         40266
like new     13863
fair          4332
new            561
salvage        294
Name: condition, dtype: int64
gas         110326
diesel        9766
hybrid        1140
other          173
electric        95
Name: fuel, dtype: int64
clean         114847
rebuilt         3669
salvage         1558
lien            1295
missing          100
parts only        31
Name: title_status, dtype: int64
automatic    111942
manual         8375
other          1183
Name: transmission, dtype: int64
4wd    53994
fwd    41552
rwd    25954
Name: drive, dtype: int64
full-size      68419
mid-size       36041
compact        15017
sub-compact     2023
Name: size, dtype: int64


In [12]:
non_numeric_columns = cars_df.select_dtypes(['object']).columns
numeric_columns = cars_df.select_dtypes(['int64', 'float64']).columns

In [13]:
cars_df = cars_df.categorize(columns=non_numeric_columns)
de = DummyEncoder()
trn_cars_df = de.fit_transform(cars_df)
trn_cars_df.head()

,price,year,odometer,condition_excellent,condition_good,condition_like new,condition_fair,condition_new,condition_salvage,fuel_gas,...,transmission_manual,transmission_automatic,transmission_other,drive_4wd,drive_fwd,drive_rwd,size_compact,size_mid-size,size_full-size,size_sub-compact
0,17899,2012.0,63500.0,1,0,0,0,0,0,1,...,1,0,0,1,0,0,1,0,0,0
24,4600,2008.0,110982.0,0,1,0,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
48,28000,2004.0,67348.0,0,1,0,0,0,0,1,...,0,1,0,1,0,0,0,0,1,0
57,2500,2004.0,129000.0,0,1,0,0,0,0,1,...,1,0,0,0,0,1,0,0,1,0
58,68500,2019.0,8800.0,0,0,1,0,0,0,1,...,1,0,0,1,0,0,0,0,1,0


In [14]:
trn_cars_df = trn_cars_df[trn_cars_df['price'] > 1000]
len(trn_cars_df)

111572

In [15]:
Y = trn_cars_df['price']
X = trn_cars_df.drop(['price'], axis=1)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [17]:
lr = LinearRegression(penalty='l2')
lr.fit(X_train.values.compute(), y_train.values.compute())

print("R-squared of the model in training set is: {}".format(lr.score(X_train.values.compute(), y_train.values.compute())))
print("R-squared of the model on the test set is: {}".format(lr.score(X_test.values.compute(), y_test.values.compute())))

R-squared of the model in training set is: 365199055376655.44
R-squared of the model on the test set is: 265116156562.18027
